<a href="https://colab.research.google.com/github/shouhardik/Alzheimers-early-detection/blob/main/Alzeimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import os
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.model_selection import train_test_split
from keras import regularizers, optimizers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint , LearningRateScheduler
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_curve, auc,confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='drive/MyDrive/Datasets/Alzheimer_sDataset'

In [ ]:
content=os.listdir('/content/drive/MyDrive/Datasets/Alzheimer_s Dataset')

In [ ]:
print(content)

['train', 'test']


In [ ]:
train_path='/content/drive/MyDrive/Datasets/Alzheimer_s Dataset/train'
test_path='/content/drive/MyDrive/Datasets/Alzheimer_s Dataset/test'
train_content=os.listdir(train_path)
test_content=os.listdir(test_path)

In [ ]:
class_train = sorted(train_content)
class_test = sorted(test_content)

In [ ]:
print(train_content)
print(test_content)

['VeryMildDemented', 'NonDemented', 'ModerateDemented', 'MildDemented']
['MildDemented', 'VeryMildDemented', 'ModerateDemented', 'NonDemented']


In [ ]:
df1=pd.DataFrame(columns=("image", "labels"))
for i in class_train:
  for j in os.listdir(train_path+'/'+i):
    df1.loc[len(df1.index)]=j,i
df1.to_csv("csv_file1")

df2=pd.DataFrame(columns=("image", "labels"))
for i in class_test:
  for j in os.listdir(test_path+'/'+i):
    df2.loc[len(df2.index)]=j,i
df2.to_csv("csv_file2")


In [ ]:
df1.head()
df2.head()

,image,labels
0,27 (21).jpg,MildDemented
1,26 (22).jpg,MildDemented
2,27 (16).jpg,MildDemented
3,27 (14).jpg,MildDemented
4,27 (2).jpg,MildDemented


In [ ]:
# Combine the two csv files
df=pd.concat([df1, df2], ignore_index=False)
print(df["labels"].value_counts())
df.to_csv("csv_file_combine")
df.head()

labels
NonDemented         3200
VeryMildDemented    2240
MildDemented         896
ModerateDemented      64
Name: count, dtype: int64


,image,labels
0,mildDem171.jpg,MildDemented
1,mildDem716.jpg,MildDemented
2,mildDem473.jpg,MildDemented
3,mildDem326.jpg,MildDemented
4,mildDem133.jpg,MildDemented


In [ ]:
train_df, test_df = train_test_split(df,test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
len(train_df), len(test_df), len(val_df)

(4096, 1280, 1024)

In [ ]:
import os
import shutil

# Define the source directory containing subfolders
source_directory = '/content/drive/MyDrive/Datasets/Alzheimer_s Dataset/train/'

# Define the destination directory where images will be combined
destination_directory = '/content/img/'
if not os.path.exists(destination_directory):
  os.makedirs(destination_directory)
# List of subfolders
subfolders = ['VeryMildDemented', 'NonDemented', 'ModerateDemented', 'MildDemented']

# Iterate through each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(source_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Iterate through each image file in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Construct the source and destination paths
                source_path = os.path.join(subfolder_path, filename)
                destination_path = os.path.join(destination_directory, filename)
                # Copy or move the image file to the destination directory
                shutil.copyfile(source_path, destination_path)  # Change to shutil.move() if you want to move instead of copy

print("Images combined successfully Train.")


# Define the source directory containing subfolders
source_directory = '/content/drive/MyDrive/Datasets/Alzheimer_s Dataset/test/'

# Iterate through each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(source_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Iterate through each image file in the subfolder
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Construct the source and destination paths
                source_path = os.path.join(subfolder_path, filename)
                destination_path = os.path.join(destination_directory, filename)
                # Copy or move the image file to the destination directory
                shutil.copyfile(source_path, destination_path)  # Change to shutil.move() if you want to move instead of copy

print("Images combined successfully Test.")



Images combined successfully Train.
Images combined successfully Test.


In [ ]:
train_gen=IDG(rescale=1/255, horizontal_flip=True, rotation_range=20)
train_images = train_gen.flow_from_dataframe(dataframe=train_df, directory='/content/img', x_col="image", y_col="labels", class_mode="categorical", shuffle=True, batch_size=16, color_mode="grayscale", target_size=(176,176))
validation_images = train_gen.flow_from_dataframe(dataframe=val_df, directory='/content/img', x_col="image", y_col="labels", class_mode="categorical", shuffle=True, batch_size=16, color_mode="grayscale", target_size=(176,176))
test_images = train_gen.flow_from_dataframe(dataframe=test_df, directory='/content/img', x_col="image", y_col="labels", class_mode="categorical", shuffle=True, batch_size=16, color_mode="grayscale", target_size=(176,176))


Found 4096 validated image filenames belonging to 4 classes.
Found 1024 validated image filenames belonging to 4 classes.
Found 1280 validated image filenames belonging to 4 classes.


In [ ]:
counter = Counter(train_images.classes)
counter_val = Counter(validation_images.classes)
counter_test = Counter(test_images.classes)
print(counter.items())
print(counter_val.items())
print(counter_test.items())

dict_items([(0, 554), (2, 2050), (3, 1452), (1, 40)])
dict_items([(3, 365), (0, 141), (2, 507), (1, 11)])
dict_items([(2, 643), (3, 423), (0, 201), (1, 13)])


In [ ]:
class_counts=[554, 2050, 1452, 40]
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_images.classes), y=train_images.classes)
class_weight_dict=dict(enumerate(class_weights))

In [ ]:
class_weight_dict

{0: 1.848375451263538, 1: 25.6, 2: 0.4995121951219512, 3: 0.7052341597796143}

In [ ]:
model1=models.Sequential()
model1.add(layers.Conv2D(128, (3,3), padding='same', input_shape=(176, 176, 1)))
model1.add(layers.MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization())
model1.add(layers.Dropout(0.3))
model1.add(layers.Activation("relu"))
model1.add(layers.Conv2D(64,(3, 3),  padding='same'))
model1.add(layers.MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(layers.Dropout(0.3))
model1.add(layers.Activation("relu"))
model1.add(layers.Conv2D(32,(3, 3),  padding='same'))
model1.add(layers.MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(layers.Dropout(0.3))
model1.add(layers.Activation("relu"))
model1.add(layers.Conv2D(16,(3, 3),  padding='same'))
model1.add(layers.MaxPooling2D(pool_size=(2, 2)))
model1.add(BatchNormalization())
model1.add(layers.Dropout(0.3))
model1.add(layers.Activation("relu"))
model1.add(layers.Flatten())
model1.add(layers.Dense(512))
model1.add(layers.Dropout(0.3))
model1.add(layers.Dense(4, activation="softmax"))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 176, 176, 128)     1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 88, 88, 128)       0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 88, 88, 128)       512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 88, 88, 128)       0         
                                                                 
 activation (Activation)     (None, 88, 88, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 88, 88, 64)        7

In [ ]:

model1.save_weights('/content/new_model_weights.h5')
model1.load_weights('/content/new_model_weights.h5')

In [ ]:
model_checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, save_weights_only=True, monitor='val_accuracy', mode='max', verbose=2)
history = model1.fit(train_images, validation_data=validation_images ,callbacks=[model_checkpoint], class_weight=class_weight_dict , epochs= 100)

Epoch 1/100
256/256 [==============================] - ETA: 0s - loss: 4.5256 - accuracy: 0.3347
Epoch 1: val_accuracy improved from -inf to 0.13770, saving model to best_model_1.h5
256/256 [==============================] - 36s 105ms/step - loss: 4.5256 - accuracy: 0.3347 - val_loss: 1.8437 - val_accuracy: 0.1377
Epoch 2/100
256/256 [==============================] - ETA: 0s - loss: 3.3974 - accuracy: 0.3245
Epoch 2: val_accuracy did not improve from 0.13770
256/256 [==============================] - 27s 104ms/step - loss: 3.3974 - accuracy: 0.3245 - val_loss: 1.8061 - val_accuracy: 0.0107
Epoch 3/100
256/256 [==============================] - ETA: 0s - loss: 2.1542 - accuracy: 0.3372
Epoch 3: val_accuracy improved from 0.13770 to 0.18359, saving model to best_model_1.h5
256/256 [==============================] - 27s 107ms/step - loss: 2.1542 - accuracy: 0.3372 - val_loss: 1.5493 - val_accuracy: 0.1836
Epoch 4/100
256/256 [==============================] - ETA: 0s - loss: 1.6267 - acc

In [ ]:
test_loss, test_accuracy = model1.evaluate(test_images) # 1st

80/80 [==============================] - 4s 47ms/step - loss: 1.2935 - accuracy: 0.3703


In [ ]:
model1.evaluate(train_images) , model1.evaluate(validation_images) #1st


64/64 [==============================] - 4s 67ms/step - loss: 1.3423 - accuracy: 0.3320


([1.2803293466567993, 0.37841796875], [1.3422502279281616, 0.33203125])

In [ ]:
true_test_labels = test_images.classes
test_pred = model1.predict(test_images)

test_pred_classes = np.argmax(test_pred, axis=1)
confusion_Matrix = confusion_matrix(true_test_labels, test_pred_classes)
sns.heatmap(confusion_Matrix, annot=True, cmap='Blues',fmt='.3g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
test_report = classification_report(true_test_labels, test_pred_classes)
print(test_report)

In [ ]:
import joblib

# Assuming you have a trained model named 'model'
joblib.dump(model1, 'model1.sav')

['model1.sav']

In [ ]:
import pickle

# Assuming you have a trained model named 'model'
with open('model11.sav', 'wb') as f:
    pickle.dump(model1, f)
#joblib.load() or pickle.load()